In [6]:
from openai import OpenAI
import os
from datasets import load_dataset
from pinecone import Pinecone
from tqdm.auto import tqdm

os.environ["OPENAI_API_KEY"] = ""

In [15]:
pc = Pinecone(api_key = "")

index = pc.Index("openai")

In [8]:
data = load_dataset("jamescalam/ml-qa", split = "train")
data

Dataset({
    features: ['docs', 'category', 'thread', 'href', 'question', 'context', 'marked'],
    num_rows: 6165
})

In [9]:
data[100]

{'docs': 'huggingface',
 'category': 'Beginners',
 'thread': 'Training stops when I try Fine-Tune XLSR-Wav2Vec2 for low-resource ASR',
 'href': 'https://discuss.huggingface.co/t/training-stops-when-i-try-fine-tune-xlsr-wav2vec2-for-low-resource-asr/8981',
 'question': 'Hi,\nI’m learning Wav2Vec2 according the blog link:\n  \n\n      huggingface.co\n  \n\n  \n    \n\nFine-Tune XLSR-Wav2Vec2 for low-resource ASR with 🤗 Transformers 1\n\n\n\n  \n\n  \n    \n    \n  \n\n  \n\n\nAnd I download the ipynb file and try run it locally.\nFine_Tune_XLSR_Wav2Vec2_on_Turkish_ASR_with_🤗_Transformers.ipynb\nAll looks file but when I run trainer.train(), it seems stop after a while, and it generate some log files under the folder wav2vec2-large-xlsr-turkish-demo, I send the screen shot to you as following:\n\n2021-08-05 17-05-36 的屏幕截图1063×410 35 KB\n\nI don’t know how to open the file events.out.tfevents.1628152300.tq-sy.129248.2, what’s the problem and how can I debug of it? please help.\nThanks a lo

In [10]:
len(data)

6165

In [17]:
text = [
    f"Thread title: {x['thread']}\n\n"+
    f"Question asked: {x['question']}\n\n"+ 
    f"Given Answer: {x['context']}" for x in data]

text[100]

'Thread title: Training stops when I try Fine-Tune XLSR-Wav2Vec2 for low-resource ASR\n\nQuestion asked: Hi,\nI’m learning Wav2Vec2 according the blog link:\n  \n\n      huggingface.co\n  \n\n  \n    \n\nFine-Tune XLSR-Wav2Vec2 for low-resource ASR with 🤗 Transformers 1\n\n\n\n  \n\n  \n    \n    \n  \n\n  \n\n\nAnd I download the ipynb file and try run it locally.\nFine_Tune_XLSR_Wav2Vec2_on_Turkish_ASR_with_🤗_Transformers.ipynb\nAll looks file but when I run trainer.train(), it seems stop after a while, and it generate some log files under the folder wav2vec2-large-xlsr-turkish-demo, I send the screen shot to you as following:\n\n2021-08-05 17-05-36 的屏幕截图1063×410 35 KB\n\nI don’t know how to open the file events.out.tfevents.1628152300.tq-sy.129248.2, what’s the problem and how can I debug of it? please help.\nThanks a lot.\n\nGiven Answer: It probably stops cause u don’t have enough resources to run the script, I recommend trying to run the script on google collab'

In [12]:
client = OpenAI()

MODEL = "text-embedding-3-small"  
q = "sample text"      
res = client.embeddings.create(
            input= q,
            model = MODEL
        )
res

CreateEmbeddingResponse(data=[Embedding(embedding=[0.023119593039155006, 0.005564036313444376, 0.019725345075130463, -0.0372176319360733, -0.015676066279411316, -0.0705646350979805, -0.002919723279774189, -0.012773091904819012, 0.02368530072271824, 0.0010355806443840265, 0.03617553785443306, -0.045703254640102386, -0.05409955233335495, 0.010763339698314667, 0.05972685664892197, 0.052878811955451965, -0.004979719407856464, -0.004518220666795969, -0.03310880810022354, 0.02929772064089775, 0.02313448116183281, 0.04085007309913635, 0.0029699672013521194, 0.01447765901684761, -0.0006057169521227479, -0.02882133424282074, -0.020320827141404152, 0.0021269877906888723, 0.06967140734195709, -0.05332542583346367, -0.01298151072114706, -0.05195581540465355, 0.004391680471599102, -0.03855746611952782, -0.00595482112839818, 0.058357249945402145, 0.03382338583469391, -0.0073877000249922276, 0.0053072343580424786, -0.06955230981111526, -0.0031076723244041204, -0.013822629116475582, 0.0372771807014942

In [14]:
len(res.data[0].embedding)

1536

In [18]:
client = OpenAI()

MODEL = "text-embedding-3-small"
batch_size = 32

for i in tqdm(range(0, len(text), batch_size)):
    try:    
        i_end = min(i+batch_size, len(text))

        ids_batch = [str(r) for r in range(i, i_end)]
        text_batch = text[i:i_end]
        res = client.embeddings.create(
            input= text_batch,
            model = MODEL
        )
        embeds = [e.embedding for e in res.data]
        meta_batch = [data[x] for x in range(i, i_end)]

        to_upsert = list(zip(ids_batch, embeds, meta_batch))
        index.upsert(vectors = to_upsert)

    except Exception as e:
        print(e)
        continue

 17%|█▋        | 33/193 [10:00<41:03, 15.40s/it]  

Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens, however you requested 8769 tokens (8769 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}


 20%|██        | 39/193 [12:17<55:36, 21.66s/it]

Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens, however you requested 11750 tokens (11750 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}


 41%|████      | 79/193 [25:36<40:37, 21.38s/it]

...

Error code: 429 - {'error': {'message': 'Rate limit reached for text-embedding-3-small in organization org-3ws1C4bzID7ECYWwncSma1J7 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}


 59%|█████▊    | 113/193 [36:55<28:47, 21.59s/it]

Error code: 429 - {'error': {'message': 'Rate limit reached for text-embedding-3-small in organization org-3ws1C4bzID7ECYWwncSma1J7 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}


 59%|█████▊    | 113/193 [37:16<26:23, 19.79s/it]


KeyboardInterrupt: 

In [19]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.03392,
 'namespaces': {'': {'vector_count': 3392}},
 'total_vector_count': 3392}

In [34]:
query = "Can we build a pytorch model in tensorflow?"

res = client.embeddings.create(
    input = query,
    model = MODEL
)
res.data

[Embedding(embedding=[-0.003338051261380315, -0.046663276851177216, -0.0020298624876886606, -0.022915001958608627, 0.013788681477308273, -0.014284676872193813, -0.0016665456350892782, 0.007360576651990414, -0.047377511858940125, 0.01710193231701851, -0.010564709082245827, 0.015217148698866367, 0.02116909809410572, -0.012905809096992016, -0.02654569037258625, -0.023966513574123383, 0.01934383250772953, -0.016060341149568558, 0.037477437406778336, 0.03700128197669983, -0.010763106867671013, 0.0004070264985784888, 0.03432290256023407, 0.0011767498217523098, -0.01065398845821619, 0.011447581462562084, -0.017945125699043274, 0.0010180312674492598, -0.009810795076191425, 0.03682272136211395, -0.03505697846412659, -0.02243884652853012, -0.065550796687603, 0.008223609067499638, 0.014304516837000847, -0.03362850844860077, 0.02652585133910179, 0.018599839881062508, 0.0008035131031647325, 0.0517025962471962, -0.009577677585184574, 0.027874959632754326, -0.007325856946408749, 0.037933751940727234,

In [35]:
xq = res.data[0].embedding
res = index.query(vector = xq, top_k = 2, include_metadata= True)

In [36]:
res

{'matches': [{'id': '1838',
              'metadata': {'category': '🤗Transformers',
                           'context': 'Hi @gruffgoran, your use cases sound '
                                      'like a perfect match for the ONNX 1 '
                                      'format \n'
                                      'Having said that, you might be able to '
                                      'get a quick win by trying something '
                                      'like the following (see docs 1):\n'
                                      'tf_model = '
                                      'TFBertForSequenceClassification.from_pretrained("KB/bert-base-swedish-cased", '
                                      'from_pt=True)\n'
                                      '\n'
                                      'From here you can then run inference / '
                                      'fine-tune etc using TensorFlow.\n'
                                      'If you want to g

In [41]:
res["matches"][0]["metadata"]["context"]

'Hi @gruffgoran, your use cases sound like a perfect match for the ONNX 1 format \nHaving said that, you might be able to get a quick win by trying something like the following (see docs 1):\ntf_model = TFBertForSequenceClassification.from_pretrained("KB/bert-base-swedish-cased", from_pt=True)\n\nFrom here you can then run inference / fine-tune etc using TensorFlow.\nIf you want to go the ONNX route, the idea would be to convert PyTorch → ONNX and then load the ONNX model in TensorFlow. Details on doing the conversion can be found here: Exporting transformers models — transformers 4.3.0 documentation 2'

In [43]:
limit = 3750

contexts = [x["metadata"]["context"] for x in res["matches"]]

prompt_start = (
    "Answer the question based on the context below\n\n"+
    "Context:\n"
)
prompt_end = (
    f"\n\n Question:{query}\n Answer:"
)

for i in range(1, len(contexts)):
    if len("\n\n---\n\n".join(contexts[:i]))>= limit:
        prompt = (
            prompt_start +
            "\n\n---\n\n".join(contexts[:i-1])+
            prompt_end
        )
        break 
    elif i == len(contexts)-1:
        prompt = (
            prompt_start +
            "\n\n---\n\n".join(contexts)+
            prompt_end
        )

In [49]:
res = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt = prompt,
    temperature = 0,
    max_tokens = 400
)

In [61]:
res.choices[0].text.strip()

'Yes, it is possible to build a PyTorch model in TensorFlow by converting it to ONNX and then loading the ONNX model in TensorFlow.'